In [42]:
import pandas as pd
import time
import torch
import re

**Функция, реализующая код Цезаря.**

In [43]:
# Функция шифрования с тремя параметрами: текст, ключ, язык
def ceaser_cipher(user, key, lang):
    # Переменная результата шифрования; переменная, определяющая верхний и нижний регистр
    res, n = [], ""

    # Проверка пользователем выбранного языка

    # Проверка выбран ли русский язык (регистр букв, вводимых пользователем, не важен)
    if lang.lower() in ["русский", "russian"]:
        # Двум переменным присваиваются русская азбука нижнего и верхнего регистра соответственно
        dictionary, dictionary_upper = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя", "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
    # Проверка выбран ли английский язык язык (регистр букв, вводимых пользователем, не важен)
    elif lang.lower() in ["английский", "english"]:
        # Двум переменным присваиваются английской азбука нижнего и верхнего регистра соответственно
        dictionary, dictionary_upper = "abcdefghijklmnopqrstuvwxyz", "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    else:
        return "Такого языка нет в опции"

    # Цикл проверки, где каждую итерацию будет обрабатываться один символ из текста последовательно
    for i in range(len(user)):
        # Проверка символа на верхний или нижний регистр

        # Принадлежит ли символ нижнему регистру
        if user[i] in dictionary:
            n = dictionary
        # Принадлежит ли символ верхнему регистру
        elif user[i] in dictionary_upper:
            n = dictionary_upper
        # Символ не принадлежит ни нижнему ни верхнему регистру (символ не является буквой)
        else:
            res.append(user[i])

        # Если символ есть в списке n (является буквой), то будет происходить его зашифровка
        if user[i] in n:
            # Цикл перебора азбуки
            for j in range(len(n)):
                # Если порядковый номер буквы + ключ находятся  в диапазоне от 0 до конца азбуки
                # и если буква из текста совпадает с буквой из азбуки, то:
                if 0 <= j + key < len(n) and user[i] == n[j]:
                    # В результат добавляется буква со сдвигом key (зашифрованная буква)
                    res.append(n[j + key])
                # Если порядковый номер буквы + ключ выходит из диапазона азбуки, превышая его
                # и если буква из текста совпадает с буквой из азбуки, то:
                elif j + key >= len(n) and user[i] == n[j]:
                    # В результат добавляеться буква со сдвигом key,
                    # при этом преводя порядковый номер буквы к диапазону азбуки (зашифрованая буква)
                    res.append(n[(1 - j - key) % (len(n) - 1)])
                # Если порядковый номер буквы + ключ выходит из диапазона азбуки, недотягивает до него
                # и если буква из текста совпадает с буквой из азбуки, то:
                elif j + key < 0 and user[i] == n[j]:
                    # В результат добавляется буква со сдвигом key,
                    # при этом приводя порядковый номер буквы к диапазону азбуки (зашифрованная буква)
                    res.append(n[(j + key) % len(n)])

    # Функция возвращает зашифрованный текст
    return ''.join(res)

**Загружаем текст для обучения модели.**

In [44]:
with open('pushkin_stihi2.txt', encoding='cp1251') as f:
    text_orig = f.read().lower()

print('length:', len(text_orig))

length: 208312


In [45]:
keys = [2, 3, 4, 5, 6, 7, 8, 9, 10]

text_X = []

kid = 0
for ph in text_orig.split('\n'):
    if not (type(ph) is str): continue
    ls = []
    for c in re.sub('\s+', ' ', ceaser_cipher(ph.strip(), key=keys[kid], lang="русский")):
        if c in ['.', ',', '-', ':', ';', '"', '!', '?']: continue
        ls.append(c)
    text_X.append(ls)

    kid += 1

    if kid >= len(keys): kid = 0

text_X[23]

['ц',
 'х',
 'н',
 'п',
 'и',
 'п',
 'с',
 'ж',
 ' ',
 'ф',
 'ж',
 ' ',
 'у',
 'х',
 'л',
 'у',
 ' ',
 'ц',
 'х',
 'к',
 'и',
 'х',
 'ч',
 'ь',
 'л']

In [46]:
text_y = [[c for c in re.sub('\s+', ' ', ph.strip()) if not (c in ['.', ',', '-', ':', ';', '"', '!', '?'])] for ph in text_orig.split('\n') if type(ph) is str]

text_y[23]

['п',
 'о',
 'ж',
 'и',
 'в',
 'и',
 'к',
 'а',
 ' ',
 'н',
 'а',
 ' ',
 'м',
 'о',
 'е',
 'м',
 ' ',
 'п',
 'о',
 'д',
 'в',
 'о',
 'р',
 'ь',
 'е']

## Делаем массив с данными

In [47]:
MAX_LEN = 30

In [48]:
CHARS = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [49]:
X = torch.zeros((len(text_X), MAX_LEN), dtype=int)
for i in range(len(text_X)):
    for j, w in enumerate(text_X[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

y = torch.zeros((len(text_y), MAX_LEN), dtype=int)
for i in range(len(text_y)):
    for j, w in enumerate(text_y[i]):
        if j >= MAX_LEN:
            break
        y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [50]:
X[23:25], y[23:25]

(tensor([[24, 23, 15, 17, 10, 17, 19,  8, 34, 22,  8, 34, 21, 23, 13, 21, 34, 24,
          23, 12, 10, 23, 25, 30, 13,  0,  0,  0,  0,  0],
         [24, 20,  9, 16, 18, 34, 27, 11, 24, 14, 34, 29, 27, 14, 26, 13, 18, 14,
          34, 18, 34, 25, 26, 24, 11, 24, 26, 29, 14,  0]]),
 tensor([[17, 16,  8, 10,  3, 10, 12,  1, 34, 15,  1, 34, 14, 16,  6, 14, 34, 17,
          16,  5,  3, 16, 18, 30,  6,  0,  0,  0,  0,  0],
         [16, 12,  1,  8, 10, 34, 19,  3, 16,  6, 34, 21, 19,  6, 18,  5, 10,  6,
          34, 10, 34, 17, 18, 16,  3, 16, 18, 30,  6,  0]]))

**Модель.**

In [51]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [68]:
model = Network()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

**Обучаем модель.**

In [69]:
for ep in range(900):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        X_batch = X[i * 100:(i + 1) * 100]
        Y_batch = y[i * 100:(i + 1) * 100].flatten()

        optimizer.zero_grad()
        answers = model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

     

    if ep%5 == 0: 
        print("\nEpoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        #s = generate_sentence()
        #print(s)
    else:
        print(f"\rEpoch {ep}, loss: {train_loss / train_passed:.3f}", end='') 


Epoch 0. Time: 1.295, Train loss: 2.487
Epoch 4, loss: 1.713
Epoch 5. Time: 1.208, Train loss: 1.649
Epoch 9, loss: 1.457
Epoch 10. Time: 1.250, Train loss: 1.422
Epoch 14, loss: 1.328
Epoch 15. Time: 1.199, Train loss: 1.312
Epoch 19, loss: 1.265
Epoch 20. Time: 1.214, Train loss: 1.256
Epoch 24, loss: 1.224
Epoch 25. Time: 1.204, Train loss: 1.216
Epoch 29, loss: 1.165
Epoch 30. Time: 1.207, Train loss: 1.150
Epoch 34, loss: 1.101
Epoch 35. Time: 1.222, Train loss: 1.091
Epoch 39, loss: 1.052
Epoch 40. Time: 1.199, Train loss: 1.044
Epoch 44, loss: 1.012
Epoch 45. Time: 1.192, Train loss: 1.005
Epoch 49, loss: 0.978
Epoch 50. Time: 1.206, Train loss: 0.971
Epoch 54, loss: 0.947
Epoch 55. Time: 1.209, Train loss: 0.941
Epoch 59, loss: 0.919
Epoch 60. Time: 1.185, Train loss: 0.913
Epoch 64, loss: 0.890
Epoch 65. Time: 1.399, Train loss: 0.885
Epoch 69, loss: 0.863
Epoch 70. Time: 1.192, Train loss: 0.857
Epoch 74, loss: 0.836
Epoch 75. Time: 1.695, Train loss: 0.831
Epoch 79, loss: 0

**Проверка модели.**

In [70]:
def predict_sentence(sentence):
    ret = []

    x = torch.zeros((1, len(sentence)), dtype=int)
    
    for j, w in enumerate(sentence):
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
        
    o = model(x)

    for i in range(len(sentence)):
        w = torch.argmax(o[-1, i, :], keepdim=True)
        ww = INDEX_TO_CHAR[w]
        
        if ww == 'none':
            break

        ret.append(ww)
        
    return ''.join(ret)

In [71]:
cnt_start = 30
cnt = 10

In [72]:
c = 0
for ph in text_orig.split('\n'):
    print(ph)
    c += 1
    if c > cnt: break

сказка о попе и о работнике его балде
жил-был поп,
толоконный лоб.
пошел поп по базару
посмотреть кой-какого товару.
навстречу ему балда
идет, сам не зная куда.
"что, батька, так рано поднялся?
чего ты взыскался?"
поп ему в ответ: "нужен мне работник:
повар, конюх и плотник.


In [75]:
c = 0
for ph in text_orig.split('\n'):
    encripted_text = ceaser_cipher(ph.strip(), key=3, lang="русский")  # Зашифрованный текст
    etxt = ''.join([c for c in re.sub('\s+', ' ', encripted_text) if not (c in ['.', ',', '-', ':', '"', '!', '?'])])
    print(predict_sentence(etxt))                                      # Расшифровываем текст обученной моделью                    
    c += 1
    if c > cnt: break

плазка о попе и о работнике его балде
вдеаыл поп
нолоконнюй моб
иояел поп по базару
иосмотреть койкакого товару
иавстречу ему балда
идет сам не зная куда
сто батька так рано поднялся
севн ты взюскался
иоп длу в ответ нужен мне работник
иовар коных и плотник


In [74]:
for key in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    print("key =", key, ":")
    encripted_text = ceaser_cipher("звездное небо над морем", key=key, lang="русский")
    print(encripted_text)
    print(predict_sentence(encripted_text))

key = 2 :
йджйёпрж пжгр пвё ортжо
ввеебное небо над морем
key = 3 :
кезкжрсз рздс ргж псузп
ивездное небо над морем
key = 4 :
лёилзсти сиет сдз ртфир
идездное небо над морем
key = 5 :
мжймитуй тйёу теи сухйс
иеездное небо над морем
key = 6 :
нзкнйуфк укжф уёй тфцкт
ивездлое небо над морем
key = 7 :
оилокфхл флзх фжк ухчлу
иееждное небо над морем
key = 8 :
пймплхцм хмиц хзл фцшмф
ивеиеное небо над морем
key = 9 :
ркнрмцчн цнйч цим хчщнх
иелодное небо над морем
key = 10 :
слоснчшо чокш чйн цшъоц
иилиднюе небо над морем


Выводы: Модель почти расшифровывает зашифрованный кодом Цезаря с разным смещением текст.